In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse 
import pandas as pd 
import os 
import numpy as np
import torch 
from collections import defaultdict
import pickle

from law import ScalingLaw

# Functions

In [3]:
def load_break_losses(dirs, slice_list, task_name):
    files_to_exclude = ['trace', 'val_inputs', 'labels', 'proportions', 'emb', 'rouge', 'generations', 'gradient', 'acc']    
    df_all = pd.DataFrame() 

    for dir in dirs:
        files = os.listdir(dir)
        files = [os.path.join(dir, f) for f in files] 
        files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

        for file in files:
            if ".log" in file or task_name not in file:
                continue 

            if any(skill not in file for skill in slice_list):
                continue 

            if "break" not in file:
                continue

            break_steps = int(file.split("break_")[-1].split("_")[0])
           
            method = file.split("/")[-1]

            weight_str = method.split("weights_")[-1].split("_")[0]
            a, b, c = [float(f"0.{weight}") for weight in weight_str.split("0.")[1:]]

            print(a, b, c)


            runs = os.listdir(file)
            for run in runs:

                if "test_" in run:
                    continue

                if any([exclude_file in run for exclude_file in files_to_exclude]):
                    continue 

                seed = int(run.split("seed_")[-1].split("_")[0])
                checkpoint = int(run.split("-")[-1].split(".")[0])


                path = os.path.join(file, run)

                df = pd.read_pickle(path)

                df = df.rename(columns={"task_idx": "skill", "task_loss": "loss"})

                df["method"] = method
                df["seed"] = seed
                df["checkpoint"] = checkpoint
                df["break_steps"] = break_steps
                df["p1"] = a 
                df["p2"] = b


                df.set_index("checkpoint", inplace=True)


                df_all = pd.concat([df_all, df])


    df_all = df_all.sort_values(by=["checkpoint", "p1", "seed"])
    return df_all


In [4]:
def load_resume_losses(dirs, slice_list, task_name):
    files_to_exclude = ['trace', 'val_inputs', 'labels', 'proportions', 'emb', 'rouge', 'generations', 'gradient', 'acc']    
    df_all = pd.DataFrame() 

    for dir in dirs:
        files = os.listdir(dir)
        files = [os.path.join(dir, f) for f in files] 
        files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

        for file in files:
            if ".log" in file or task_name not in file:
                continue 

            if any(skill not in file for skill in slice_list):
                continue 

            if "resume" not in file:
                continue

            old_weight_str = file.split("resume_")[-1].split("_")[0]
            old_a, old_b, old_c = [float(f"0.{weight}") for weight in old_weight_str.split("0.")[1:]]


            method = file.split("/")[-1]

            new_weight_str = method.split("weights_")[-1].split("_")[0]
            new_a, new_b, new_c = [float(f"0.{weight}") for weight in new_weight_str.split("0.")[1:]]


            break_steps = int(file.split(f"resume_{old_weight_str}_")[-1].split("_")[0])


            runs = os.listdir(file)
            for run in runs:

                if "test_" in run:
                    continue

                if any([exclude_file in run for exclude_file in files_to_exclude]):
                    continue 

                seed = int(run.split("seed_")[-1].split("_")[0])
                checkpoint = int(run.split("-")[-1].split(".")[0])


                path = os.path.join(file, run)

                df = pd.read_pickle(path)

                df = df.rename(columns={"task_idx": "skill", "task_loss": "loss"})

                df["method"] = method
                df["seed"] = seed
                df["checkpoint"] = checkpoint
                df["break_steps"] = break_steps
                df["new_p1"] = new_a 
                df["new_p2"] = new_b 
                df["new_p3"] = new_c 
                df["p1"] = old_a 
                df["p2"] = old_b
                df["p3"] = old_c


                df.set_index("checkpoint", inplace=True)


                df_all = pd.concat([df_all, df])


    df_all = df_all.sort_values(by=["checkpoint", "p1", "new_p1", "seed"])
    return df_all


In [5]:
def calculate_r_squared(actuals, predictions):
    actuals, predictions = actuals.numpy(), predictions.numpy()
    # Calculate the total sum of squares
    total_sum_of_squares = np.sum((actuals - np.mean(actuals)) ** 2)
    # Calculate the residual sum of squares
    residual_sum_of_squares = np.sum((actuals - predictions) ** 2)
    # Calculate R-squared
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)
    return r_squared


### mixing law math

The generic form of the dynamics is $L_i^t(p) = c_i + \sum_{j=1}^m A_{ij}^t p_j^t$. For $m=3$, we only need two parameters per row in $A$, so we can rewrite as
\begin{align}
L_i^t(p) &= c_i + A_{i1}^t p_1^t + A_{i2}^t p_2^t + A_{i3}^t (1 - p_1^t - p_2^t) \\
&= c_i^t + A_{i3}^t + (A_{i1}^t - A_{i3}^t) p^t_1 + (A_{i2}^t - A_{i3}^t) p^t_2 \\
&= c_i^t + A_{i1}^t p_1^t + A_{i2}^t p_2^t
\end{align}

where $c_i^t = c_i^t + A_{i3}^t$, $A_{i1}^t = A_{i1}^t - A_{i3}^t,$ and $A_{i2}^t = A_{i2}^t - A_{i3}^t$.

That is, each regression involves 3 parameters.

In [6]:
def mixing_law(x, param):
    # one set of params per skill
    c_i = param[0]
    t_i = param[1:]
    result = c_i + torch.matmul(x[:, :2], t_i)
    return result

def init_params_law(idx, num_domains=3):
    for c_i in np.linspace(0.5, 5, 10):
        for _ in range(30):
            ts = [-np.random.rand() if i == idx else np.random.rand() * 0.1 for i in range(num_domains-1)]
            yield [c_i] + ts

In [ ]:
def make_individual_xy(df_resume, skill, break_steps, p1):
    x = []
    y = []

    df_resume_subset = df_resume[(df_resume.break_steps == break_steps) & (df_resume.p1 == p1) & (df_resume.skill == skill)]
    df_resume_subset = df_resume_subset.loc[df_resume_subset.index.max()]
    
        
    x = df_resume_subset[['new_p1', 'new_p2', 'new_p3']].values
    y = df_resume_subset['loss'].values

    return x, y

### mixing law (full) math

The generic form of the dynamics is $L_i^t(p) = L_i^{t-1}(p) + \sum_{j=1}^m A_{ij}^t p_j^t$. For $m=3$, we model all 3 parameters per row in $A$.

In [ ]:
# This mixing law has an extra degree of freedom, i.e, we parameterize the full A matrix
def mixing_law_full(x, param):
    result = torch.matmul(x, param)
    return result

def init_params_law_full(idx, num_domains=3):
    #for c_i in np.linspace(0.5, 5, 10):
    for _ in range(30):
        ts = [-np.random.rand() if i == idx else np.random.rand() * 0.1 for i in range(num_domains)]
        yield ts

In [9]:
def make_individual_xy_full(df_break, df_resume, skill, break_steps, p1):
    x = []
    y = []

    df_break_subset = df_break[(df_break.break_steps == break_steps) & (df_break.p1 == p1) & (df_break.skill == skill)]
    df_break_subset = df_break_subset.loc[df_break_subset.index.max()]

    df_resume_subset = df_resume[(df_resume.break_steps == break_steps) & (df_resume.p1 == p1) & (df_resume.skill == skill)]
    df_resume_subset = df_resume_subset.loc[df_resume_subset.index.max()]
    
        
    loss_0 = df_break_subset.loss
    x = df_resume_subset[['new_p1', 'new_p2', 'new_p3']].values
    y = df_resume_subset['loss'].values - loss_0

    return x, y

# Arxiv, Books, Stackexchange

In [ ]:
dirs = ["../output/09112024/", "../output/09122024/"] # INCLUDE YOUR RUN OUTPUT DIRECTORIES HERE
task_name = "slimpj"
slice_list = ["arxiv", "book", "stackexchange"]
df_break = load_break_losses(dirs, slice_list, task_name)

In [11]:
skills = sorted(df_break.skill.unique())

In [12]:
df_resume = load_resume_losses(dirs, skills, task_name)

In [13]:
break_steps = sorted(df_resume.break_steps.unique())
probs = sorted(df_break.p1.unique())

In [ ]:
params = {skill : {bs: {p: {} for p in probs} for bs in break_steps} for skill in skills }
x_per_skill = {skill : {bs: {p: {} for p in probs} for bs in break_steps} for skill in skills }
y_per_skill = {skill : {bs: {p: {} for p in probs} for bs in break_steps} for skill in skills }

mses = []
r2s = []
for i, skill in enumerate(skills):
    for bs in break_steps:
        for p1 in probs:
            print(skill, bs, p1)
            x, y = make_individual_xy(df_resume, skill, bs, p1)
            
            x_per_skill[skill][bs][p1] = x
            y_per_skill[skill][bs][p1] = y
                        
            law = ScalingLaw(mixing_law)
            p = law.fit(x, y, init_params_law(i, num_domains=len(skills)), max_step=100, delta=0.02)
            params[skill][bs][p1] = p # param

            prediction_train = mixing_law(torch.tensor(x, dtype=torch.float), torch.tensor(p, dtype=torch.float))
            mse_train = torch.nn.functional.mse_loss(prediction_train, torch.tensor(y, dtype=torch.float)).item()
            r2_train = calculate_r_squared(torch.tensor(y), torch.tensor(prediction_train))

            mses.append(mse_train)
            r2s.append(r2_train)


            print(f"MSE: {mse_train}, R2: {r2_train}")


mses = np.array(mses)
r2s = np.array(r2s)

print(mses.mean(), mses.std())
print(r2s.mean(), r2s.std())


In [ ]:
params = {skill : {bs: {p: {} for p in probs} for bs in break_steps} for skill in skills }
x_per_skill = {skill : {bs: {p: {} for p in probs} for bs in break_steps} for skill in skills }
y_per_skill = {skill : {bs: {p: {} for p in probs} for bs in break_steps} for skill in skills }

mses = []
r2s = []
for i, skill in enumerate(skills):
    for bs in break_steps:
        for p1 in probs:
            print(skill, bs, p1)
            x, y = make_individual_xy_full(df_break, df_resume, skill, bs, p1)
            
            x_per_skill[skill][bs][p1] = x
            y_per_skill[skill][bs][p1] = y
                        
            law = ScalingLaw(mixing_law_full)
            p = law.fit(x, y, init_params_law_full(i, num_domains=len(skills)), max_step=100, delta=0.02)
            params[skill][bs][p1] = p # param

            prediction_train = mixing_law_full(torch.tensor(x, dtype=torch.float), torch.tensor(p, dtype=torch.float))
            mse_train = torch.nn.functional.mse_loss(prediction_train, torch.tensor(y, dtype=torch.float)).item()
            r2_train = calculate_r_squared(torch.tensor(y), torch.tensor(prediction_train))

            mses.append(mse_train)
            r2s.append(r2_train)


            print(f"MSE: {mse_train}, R2: {r2_train}")


mses = np.array(mses)
r2s = np.array(r2s)

print(mses.mean(), mses.std())
print(r2s.mean(), r2s.std())


In [54]:
import pickle 

with open("./law_results/arxiv_books_stackexchange/params_dynamic.pkl", "wb") as f:
    pickle.dump(params, f)

